In [ ]:
import os, h5py, numpy as np
import illustris_python as il
import yt
import yt.units as ytu
from emis import emissivity
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c
from plot_utils import quiver_from_components

In [ ]:
with h5py.File('mw_tng_sample_catalog.hdf5') as f:
    mw_id = np.array(f['SubfindID'])

In [ ]:
#outpath = './cutouts/TNG50_snap099_fof000020_gas_sphere_500kpc.hdf5'
subid = mw_id[5]
outpath = f'/home/cj535/palmer_scratch/TNG50_cutouts/MW_sample/TNG50_snap099_subid{subid:06d}_gas_sphere_500kpc.hdf5'
idx_path = outpath.replace(".hdf5", ".ytindex")
ds = yt.load(outpath, index_filename=idx_path,smoothing_factor=2)
#ds.add_deposited_particle_field(("gas","Halpha_power"), "sum")

def _Ha_cloudy(field, data):
    nH = data["gas","H_number_density"].in_units("1/cm**3").astype("float64")
    T  = data["gas","temperature"].in_units("K").astype("float64")
    Z  = data["gas","metallicity"].to("dimensionless").astype("float64")                
    # your helper returns log10 emissivity
    log_eps = emissivity("H-alpha", nH.value, Z.value, T.value, redshift=float(data.ds.current_redshift))
    eps = np.power(10.0, log_eps, dtype=np.float64) * data['gas','H_fraction'] * ytu.erg/ytu.s/ytu.cm**3
    return eps
ds.add_field(
    name=("gas", "Halpha_emissivity"),
    function=_Ha_cloudy,
    sampling_type="particle",
    units="erg/s/cm**3"
)
def _Ha_brightness(field, data):
    return data[("gas","Halpha_emissivity")] / (4*np.pi) / ytu.sr
ds.add_field(
    name=("gas", "Halpha_brightness"),
    function=_Ha_brightness,
    sampling_type="particle",
    units="erg/s/cm**3/arcsec**2"
)

def _Ha_pow(field, data):
    return data[("gas","Halpha_emissivity")] * data[("gas","cell_volume")]
ds.add_field(
    name=("gas", "Halpha_power"),
    function=_Ha_pow,
    sampling_type="particle",
    units="erg/s"
)
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
sp_small = ds.sphere(center_prop, (100.0, "kpc"))
bv = sp_small.quantities.bulk_velocity()
'''
def _dv_los(field,data):
    v = data[("gas","velocity_los")]
    ax = data.get_field_parameter("axis")
    if yt.funcs.is_sequence(ax):
        # Make sure this is a unit vector
        ax /= np.sqrt(np.dot(ax, ax))
        v0 = bv[0]*ax[0] + bv[1]*ax[1] + bv[2]*ax[2]
        v0 = v0
    elif ax in [0,1,2]:
        v0 = bv[ax]
    else:
        raise NeedsParameter(["axis"])
    return v - v0
ds.add_field(
    name=("gas", "dv_los"),
    function=_dv_los,
    units="km/s",
    sampling_type="particle"
)
'''

def _mask(lo, hi):
    def _f(field, data):      
        dv = data[("gas","velocity_los")].to_value("km/s")
        return ((dv >= lo) & (dv < hi)).astype("float64")  # 1 inside, 0 outside
    ds.add_field(
        name=("gas", f"mask_{lo:.0f}_{hi:.0f}"),
        function=_f,
        units="",
        sampling_type="particle"
    )
    def _g(field, data):
        return data[("gas","Halpha_emissivity")] * data[("gas",f"mask_{lo:.0f}_{hi:.0f}")]
    ds.add_field(
        name=("gas", f"Halpha_emissivity_{lo:.0f}_{hi:.0f}"),
        function=_g,
        units="erg/s/cm**3",
        sampling_type="particle"
    )
    def _h(field, data):
        return data[("gas","Halpha_brightness")] * data[("gas",f"mask_{lo:.0f}_{hi:.0f}")]
    ds.add_field(
        name=("gas", f"Halpha_brightness_{lo:.0f}_{hi:.0f}"),
        function=_h,
        units="erg/s/cm**3/arcsec**2",
        sampling_type="particle"
    )

    return _f, _g

_mask(-300, -100); _mask(-100, 100); _mask(100, 300)

def _coldgas(field,data):
    T  = data["gas","temperature"].to_value("K")
    return (T < 2e4).astype("float64")
ds.add_field(
    name=("gas", "cold"),
    function=_coldgas,
    units="",
    sampling_type="particle"
)
def _cold_density(field,data):
    return data[('gas','density')]*data[("gas", "cold")]
ds.add_field(
    name=("gas", "cold_density"),
    function=_cold_density,
    units="g/cm**3",
    sampling_type="particle"
)
    

ds.derived_field_list

In [ ]:
print(ds.field_info[('PartType0', 'Temperature')].get_source())

In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)

theta = np.deg2rad(15.0)
w_hat = np.array([np.sin(theta), 0.0, np.cos(theta)])  # (nx, ny, nz)
v_hat = np.array([0.0, 1.0, 0.0])  # +y points "up" on the image
u_hat = np.cross(v_hat,w_hat)

#unit_vector /= np.sqrt(np.dot(unit_vector,unit_vector)) #make sure it's unit
def _vel_u(field,data):
    ux,uy,uz = u_hat[0],u_hat[1],u_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_u"),
    function=_vel_u,
    units="km/s",
    sampling_type="particle"
)
def _vel_u_cold(field,data):
    return data['gas','velocity_u']*data['gas','cold']
ds.add_field(name=('gas','cold_velocity_u'),function=_vel_u_cold,units="km/s",sampling_type="particle")
def _momentum_density_u(field,data):
    return data['gas','velocity_u']*data['gas','density']
ds.add_field(name=('gas','momentum_density_u'),function=_momentum_density_u,units="km*g/cm**3/s",sampling_type="particle")

def _vel_v(field,data):
    ux,uy,uz = v_hat[0],v_hat[1],v_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_v"),
    function=_vel_v,
    units="km/s",
    sampling_type="particle"
)
def _vel_v_cold(field,data):
    return data['gas','velocity_v']*data['gas','cold']
ds.add_field(name=('gas','cold_velocity_v'),function=_vel_v_cold,units="km/s",sampling_type="particle")
def _momentum_density_v(field,data):
    return data['gas','velocity_v']*data['gas','density']
ds.add_field(name=('gas','momentum_density_v'),function=_momentum_density_v,units="km*g/cm**3/s",sampling_type="particle")

def _vel_w(field,data):
    ux,uy,uz = w_hat[0],w_hat[1],w_hat[2]
    vx = data['gas','relative_velocity_x']
    vy = data['gas','relative_velocity_y']
    vz = data['gas','relative_velocity_z']
    return vx*ux + vy*uy + vz*uz
ds.add_field(
    name=("gas", f"velocity_w"),
    function=_vel_w,
    units="km/s",
    sampling_type="particle"
)
def _vel_w_cold(field,data):
    return data['gas','velocity_w']*data['gas','cold']
ds.add_field(name=('gas','cold_velocity_w'),function=_vel_w_cold,units="km/s",sampling_type="particle")
def _momentum_density_w(field,data):
    return data['gas','velocity_w']*data['gas','density']
ds.add_field(name=('gas','momentum_density_w'),function=_momentum_density_w,units="km*g/cm**3/s",sampling_type="particle")



In [ ]:
width_proper = (500.0, "kpc")                 # proper
center_comov = 0.5 * ds.domain_width          # code center (comoving)
center_prop  = (center_comov.to("kpc") * ds.scale_factor)
sp_small = ds.sphere(center_prop, (50.0, "kpc"))
bv = sp_small.quantities.bulk_velocity()
sp = ds.sphere(center_prop, (500.0, "kpc"))
sp.set_field_parameter("bulk_velocity", bv)
resolution = (256,256)

field = ("gas","cold_velocity_u")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","cold_density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
p.set_log(field, False)
vel_u = img.to_value(img.units)
p.show()

field = ("gas","cold_velocity_v")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","cold_density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
p.set_log(field, False)
vel_v = img.to_value(img.units)
p.show()

field = ("gas","cold_velocity_w")
p = yt.ProjectionPlot(
    ds, w_hat, field,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    weight_field=("gas","cold_density"),
    buff_size=resolution,
    data_source=sp
)
img = p.frb[field]
vel_w = img.to_value(img.units)
p.set_log(field, False)
p.show()

field_list = [("gas","Halpha_brightness"),
              ("gas","Halpha_brightness_-300_-100"),
              ("gas","Halpha_brightness_-100_100"),
              ("gas","Halpha_brightness_100_300")]
p = yt.ProjectionPlot(
    ds, w_hat, field_list,
    center=center_prop, width=width_proper,
    north_vector=v_hat,
    method="integrate",
    buff_size=resolution,
    data_source=sp
)
HalphaL = {}
range_name = ['bolo','-300_-100','-100_100','100_300']
for i, field in enumerate(field_list):
    img = p.frb[field]
    HalphaL[range_name[i]] = img.to_value(img.units)
    p.set_zlim(field, zmin=5e-21, zmax=1e-18)

p.show()

In [ ]:
ny, nx = vel_v.shape
x = np.arange(nx)
y = np.arange(ny)
X, Y = np.meshgrid(x, y)
R = np.sqrt((X-resolution[0]/2)**2 + (Y-resolution[1]/2)**2)
pc_per_pix = 500/resolution[0]
mask = np.array(R > 30/pc_per_pix,dtype=int)


step=4
# subsample
sl = (slice(None, None, step), slice(None, None, step))
Xs, Ys = X[sl], Y[sl]
Vx, Vy, Vz = (mask*vel_u)[sl], (mask*vel_v)[sl], (mask*vel_w)[sl]
Vx /= 300
Vy /= 300


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))

Hmap = np.log10(HalphaL['bolo'])
Hmap = Hmap * mask

ax.imshow(Hmap,origin='lower',cmap='grey',vmin=-23,vmax=-18)
q = ax.quiver(Xs,Ys,Vx,Vy,Vz,cmap='RdBu_r',clim=(-300,300))
#q = ax.quiver(Xs,Ys,px,py,pz,cmap='RdBu_r',clim=(-0.05,0.05))
#cb = plt.colorbar(q, ax=ax, label='speed (|v|)')
